tensorflow图的理解

主要解释：tf.Graph() tf.get_default_graph()  g.as_default() 

In [2]:
import tensorflow as tf

1、tf.Graph()会创建一个新的图，即使使用tf.Graph().as_default() 创建的图和默认图也不是同一个图


In [52]:
print('tf default graph:', tf.get_default_graph())
print('tf.Graph() graph:', tf.Graph())

('tf default graph:', <tensorflow.python.framework.ops.Graph object at 0x7f41ca681750>)
('tf.Graph() graph:', <tensorflow.python.framework.ops.Graph object at 0x7f41f079dd10>)


2、 设置默认图， tf.get_default_graph()函数会返回目前的默认图，全局时候返回tensorflow默认的默认图

In [64]:
print('tf default graph:', tf.get_default_graph())
a = tf.constant([3])
g = tf.Graph().as_default() # 不会改变之后定义的变量的图，也不会改变默认图, 单单生成一个内容管理器，
                            #必须要加上as_default()函数，将目前的图设置为默认图，否则在使用时候会出错
                            #如果此处没有加，还可以在以后的with语句中加入
d = tf.constant([4])
with g: 
    f = tf.constant([5])
    print('f graph:', f.graph) 
    print('inside with default graph:', tf.get_default_graph()) #with里面的默认图为用户用as_default()函数指定的默认图
print('g:', g)
print('a graph:', a.graph)
print('d graph:', d.graph)
print('tf default graph:', tf.get_default_graph())# 默认图并没有被改变


('tf default graph:', <tensorflow.python.framework.ops.Graph object at 0x7f41ca681750>)
('f graph:', <tensorflow.python.framework.ops.Graph object at 0x7f41f0885290>)
('inside with default graph:', <tensorflow.python.framework.ops.Graph object at 0x7f41f0885290>)
('g:', <contextlib.GeneratorContextManager object at 0x7f41f0885b50>)
('a graph:', <tensorflow.python.framework.ops.Graph object at 0x7f41ca681750>)
('d graph:', <tensorflow.python.framework.ops.Graph object at 0x7f41ca681750>)
('tf default graph:', <tensorflow.python.framework.ops.Graph object at 0x7f41ca681750>)


3、 在with语句中，tf.get_default_graph()获取的是设定的默认图,但是在with语句之外, tf.get_default_graph()返回的依然是原来默认图，
    即with语句将图的范围限定，随着with语句的结束，图也跟着结束 这一点和session不同，session不会随着with语句的结束而结束

In [53]:
print( 'tf default graph:', tf.get_default_graph())
with tf.Graph().as_default() as g:
    print('inside with graph:', g)
    print('inside with default graph:', tf.get_default_graph())#在with语句里面设置默认图之后，
                                                               # tf.get_default_graph()返回的是用户设置的默认图
print('outside with default graph:', tf.get_default_graph()) #默认图并没有因为with语句里面设置默认图的操作而改变

('tf default graph:', <tensorflow.python.framework.ops.Graph object at 0x7f41ca681750>)
('inside with graph:', <tensorflow.python.framework.ops.Graph object at 0x7f41ca681310>)
('inside with default graph:', <tensorflow.python.framework.ops.Graph object at 0x7f41ca681310>)
('outside with default graph:', <tensorflow.python.framework.ops.Graph object at 0x7f41ca681750>)


4、定义函数时候，不要使用图，否则函数调用时候会出错，会导致不同图中的变量进行计算，这是不被允许的

In [77]:

def myadd():
    with tf.Graph().as_default as g1: #函数定义时候不要指定图，因为跨图调用函数是不被允许的，
                                      #当函数定义时候没有指定图时，函数将被装载到调用位置所在的图中， 如myadd1所示
        a = tf.constant([1])
        b = tf.constant([2])
        print('g1 graph:', g1)
        return tf.add(a,b)
   
c = myadd() #错， 不能在tensorflow默认图中调用g1图中的函数，跨图操作是不能的
d = tf.constant([6])
tf.add(c,d) #报错， 因为c是图g1中的量，d是tensorflow默认图中的量，两者不在一个图中，不能进行计算

print('c graph:', c.graph)
print('d graph:', d.graph)


print('*****************')


AttributeError: __exit__

In [79]:
def myadd1():
    a = tf.constant([1])
    b = tf.constant([2])
    return tf.add(a,b)

cc = myadd1()  # 调用位置是默认图，因此cc.graph和tf.get_default_graph()相同，都是默认图
print(cc.graph)
print(tf.get_default_graph())

with tf.Graph().as_default() as gg: 
    cc = myadd1() #此时,cc 在图gg中被调用，cc的图就是gg
    print('cc graph:', cc.graph)
    print('gg graph:', gg)
    print('inside "with" default graph:', tf.get_default_graph()) #在图gg中，所以tf.get_default_graph()返回的就是图gg
print('outside "with" default graph:', tf.get_default_graph())#with语句结束时候，退出设定图，会到默认图，这一点和session不同
                                                              #session具有延续性


('cc graph:', <tensorflow.python.framework.ops.Graph object at 0x7f41f0876d90>)
('gg graph:', <tensorflow.python.framework.ops.Graph object at 0x7f41f0876d90>)
('inside with default graph:', <tensorflow.python.framework.ops.Graph object at 0x7f41f0876d90>)
('outside with default graph:', <tensorflow.python.framework.ops.Graph object at 0x7f41ca681750>)


5、session的延续性

In [86]:
def myadd():
    a = tf.constant([1.])
    b = tf.constant([3.])
    return tf.add(a,b)

print( tf.get_default_session())
with tf.Session() as sess:
    sess.as_default()
    print(sess)
    d = sess.run(myadd())
    print(d)
print( sess ) #在with语句以为，sess依然存在,虽然已经被关闭，不能使用
print(sess.run(myadd()))#错，提示sess已经被关闭

None
[4.]


RuntimeError: Attempted to use a closed Session.


    运行多个graph需要多个session，而每个session会试图耗尽所有的计算资源，开销太大；
    graph之间没有数据通道，要人为通过python/numpy传数据；


In [21]:
#tf.get_default_graph() 返回tensorflow默认图，不会因为g.as_default()函数的使用而改变
g = tf.get_default_graph()
g2 = tf.Graph().as_default()
g3 = tf.Graph()
print(g)
print(g2)
print(g3)
print(tf.get_default_graph()) #虽然中间使用了as_default()函数，但是tensorflow默认图并没有因此而改变

print( "*********************************************")
g4 = tf.Graph()
with g4.as_default():
    c = tf.constant([3.])
    assert c.graph is g4
    print(c.graph)
    print(g4)
    print(tf.get_default_graph()) #在with语句里面， 默认graph会变为g4
g4.as_default() # 如果不包含在with语句里面，as_default() 函数无效
print(g4)
print( tf.get_default_graph())#但是在with语句外面， 默认graph变回原来tensorflow默认图

print("************************************************")

sess = tf.Session()
print(sess)
with tf.Session(graph = tf.Graph()).as_default() as sess:
    print( sess.graph)
    print(sess)
print(sess)# session 的as_default()函数会对默认session的改变会延续到with语句以外
print( sess.graph)#默认session的图也会延续到with语句以外

*********************************************
************************************************


In [ ]:
#graph返回值 可以在不同graph中使用

In [49]:
#函数定义时候，不能指定图，当函数被调用的时候才会给函数里面的变量指定图， 如果不指定为默认图
def tt():
    #with tf.Graph().as_default() as g: 
    c = tf.constant([4])
    b = tf.constant([5])
    a = tf.add(b,c)
    w = tf.Variable(tf.constant([4]))
    print(w.graph)
    print( "a's graph:")
    print(a.graph)
    #print(g)
    return a

with tf.Graph().as_default() as gg:
    print(gg)
    d = tt()#d是属于图g中的值
    e = tf.constant([5])#e是属于图gg中的
    f = tf.add(d, e) #不同图中的值不能运算
    print( "d's graph:")
    print( d.graph)
    print( e.graph)
    print( gg )
print("****************************************")
with tf.Graph().as_default() as ggg:
    dd = tt()
    print( dd.graph )
print(tf.get_default_graph())


a's graph:
d's graph:
****************************************
a's graph:
